In [1]:
import pandas as pd
import nltk
import csv
import nltk
import random
from nltk.corpus import names

nltk.download('names')


[nltk_data] Downloading package names to /Users/ivan/nltk_data...
[nltk_data]   Package names is already up-to-date!


True

In [2]:
with open('wgnd_2_0_name-gender-code.csv', mode='r', encoding='utf-8') as infile:
    reader = csv.reader(infile)
    with open('wgnd_2_0_name-gender-code_new.csv', mode='w', encoding='utf-8') as outfile:
        writer = csv.writer(outfile)
        mydict = {rows[0]:rows[2] for rows in reader}

In [3]:
def gender_features(word):
    return {'whole_name': word[:]}


labeled_names = ([(name, 'male') for name in names.words('male.txt')] +
                 [(name, 'female') for name in names.words('female.txt')])

random.shuffle(labeled_names)

featuresets = [(gender_features(n), gender) for (n, gender) in labeled_names]

train_set, test_set = featuresets[1000:], featuresets[:1000]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, train_set))

0.9605414746543779


In [4]:
df_total = pd.read_csv('2022_patent_grant_processed.csv', index_col=0)
df_total.head()

,publication_id,publication_kind,publication_date,application_id,application_date,country,inventor_name,inventor_firstname,inventor_country,invention_title,number_of_claims,attorney_name,attorney_firstname,inventor males,inventor females,inventor team size,attorney males,attorney females,attorney team size
0,US11218467,B2,20220104,US16249715,20190116,US,"['Kharidehal Chandra Mouli', 'Rathi Sumit', 'd...","['Chandra Mouli', 'Sumit', 'Gagan', 'Santosh K...","['IN', 'IN', 'IN', 'IN']",System and method for authentication as a service,20,[],[],4,0,4,0,0,0
1,US11218468,B2,20220104,US16291907,20190304,US,"['Dhamankar Shailendra', 'Babu Krishnaraj', 'S...","['Shailendra', 'Krishnaraj', 'Shelby', 'Sriniv...","['US', 'US', 'US', 'US']",MSISDN request handling for identity fraud man...,20,[],[],4,0,4,0,0,0
2,US11218469,B2,20220104,US16294787,20190306,US,"['Valites Matthew C.', 'Bollinger Jeffrey Brya...","['Matthew C.', 'Jeffrey Bryan', 'Ian Anders']","['US', 'US', 'US']",Systems and methods for controlling personal i...,12,[],[],3,0,3,0,0,0
3,US11218470,B2,20220104,US16551538,20190826,US,['Bogrett Steve'],['Steve'],['US'],Highly available web-based database interface ...,20,[],[],1,0,1,0,0,0
4,US11218471,B1,20220104,US16787605,20200211,US,"['Stapleton Jeff J.', 'Carter, Jr. Robert L.',...","['Jeff J.', 'Robert L.', 'Pierre', 'Bradford A...","['US', 'US', 'US', 'US', 'US', 'US']",Systems and methods for quantum consensus,20,[],[],6,0,6,0,0,0


In [5]:
df = df_total.groupby(['country'])['inventor females'].sum()
df.head()

country
AR      0
AT      4
AU    105
BE     18
BR     21
Name: inventor females, dtype: int64

In [6]:
test = df_total.groupby('country', as_index=False).agg({
    'inventor females': sum,
    'inventor team size': sum
})
test.head()

,country,inventor females,inventor team size
0,AR,0,4
1,AT,4,160
2,AU,105,841
3,BE,18,123
4,BR,21,91


In [7]:
df = df_total.groupby('country', as_index=False).agg({
    'inventor females': sum,
    'inventor team size': sum,
    'attorney females': sum,
    'attorney team size': sum,
})

df.head()

,country,inventor females,inventor team size,attorney females,attorney team size
0,AR,0,4,1,1
1,AT,4,160,0,17
2,AU,105,841,4,85
3,BE,18,123,2,22
4,BR,21,91,1,13


In [10]:
df['total female ratio'] = df.apply(lambda row: round((row['inventor females'] + row['attorney females']) / (row['inventor team size'] + row['attorney team size']), 2) * 100, axis=1)
df.head()

,country,inventor females,inventor team size,attorney females,attorney team size,total female ratio
0,AR,0,4,1,1,20.0
1,AT,4,160,0,17,2.0
2,AU,105,841,4,85,12.0
3,BE,18,123,2,22,14.0
4,BR,21,91,1,13,21.0
